In [1]:
type45 = "i2,i4,i5,il100,il60,il80,io,ip,p10,p11,p12,p19,p23,p26,p27,p3,p5,p6,pg,ph4,ph4.5,ph5,pl100,pl120,pl20,pl30,pl40,pl5,pl50,pl60,pl70,pl80,pm20,pm30,pm55,pn,pne,po,pr40,w13,w32,w55,w57,w59,wo"
classes = type45.split(",")

In [8]:
import os

def extract_number(filename):
    return int(filename.split('.')[0])

annotation_path = "./VOCdevkit/labels/val/"
cls_cnt = {}
gt_list = []
txt_files = os.listdir(annotation_path)
txt_files = sorted(txt_files, key=extract_number)

for txt_file in txt_files:
    with open(annotation_path + txt_file, "r") as f:
        cls_list = []
        lines = f.readlines()
        for line in lines:
            cls = int(line.strip().split()[0])
            if cls not in cls_cnt:
                cls_cnt[cls] = 1
            else:
                cls_cnt[cls] += 1
            cls_list.append(cls)

        gt_list.append(cls_list)

print(cls_cnt)
print(gt_list)

{26: 28, 16: 19, 35: 41, 37: 31, 41: 5, 7: 12, 12: 11, 25: 9, 42: 6, 31: 16, 4: 13, 6: 36, 10: 5, 20: 4, 33: 5, 14: 4, 13: 7, 43: 9, 21: 9, 34: 1, 23: 15, 29: 17, 22: 17, 3: 5, 5: 4, 2: 5, 44: 24, 39: 6, 38: 1, 30: 9, 40: 6, 8: 5, 24: 9, 18: 1, 9: 15, 28: 5, 1: 12, 11: 1, 0: 3, 15: 1, 32: 1, 36: 6, 17: 1}
[[26], [16, 26, 35], [37], [41], [7], [35, 26], [12], [25], [7], [25], [25], [42], [31, 31, 31, 31, 4, 4, 6, 6], [26], [26], [31], [31], [37, 10], [20, 33, 14, 13], [43], [21, 21, 21, 21, 21], [43], [10], [31, 31, 31, 4], [26], [26, 34], [43], [31], [23], [29], [43], [23, 22, 22, 37], [16], [23, 3, 23, 3, 22, 5, 22, 4], [16], [16], [16], [43], [16, 23, 22, 22], [16], [16], [29], [43], [16], [43], [16], [23, 22, 22], [23, 23, 22, 22, 3, 3, 5, 4], [16], [16], [16], [29], [43], [16], [23, 22, 22], [23, 22, 22, 31, 4, 5, 5, 3], [16], [16], [29], [16], [35], [35], [35], [35], [2], [2], [29], [41, 20], [20], [44, 35, 7], [7], [6], [29], [29], [29], [29, 44], [23], [21], [23, 4, 22, 4], [21]

In [6]:
from ultralytics import YOLO

model = YOLO(
    "/Users/jiangcheng/deep_learning/experiments/train/train_yolov8l_f/weights/best.pt"
)

img_path = "./VOCdevkit/images/val/"
img_list = os.listdir(img_path)
img_list = sorted(img_list, key=extract_number)

# Run inference on images
results = model([
    os.path.join(img_path, img)
    for img in img_list
])

pre_cls_list = []

# View results
for result in results:
    res = result.boxes.cls.tolist()
    res = [int(item) for item in res]
    pre_cls_list.append(res)

print(pre_cls_list)


0: 640x640 1 pl40, 1 pn, 4245.5ms
1: 640x640 1 pl40, 1 pn, 4245.5ms
2: 640x640 (no detections), 4245.5ms
3: 640x640 1 w55, 4245.5ms
4: 640x640 1 ip, 4245.5ms
5: 640x640 1 pl40, 1 pn, 4245.5ms
6: 640x640 1 p23, 1 pn, 4245.5ms
7: 640x640 1 pl50, 1 pn, 4245.5ms
8: 640x640 1 ip, 4245.5ms
9: 640x640 1 pl50, 1 pl60, 4245.5ms
10: 640x640 1 pl30, 1 pl80, 1 pn, 4245.5ms
11: 640x640 1 w57, 4245.5ms
12: 640x640 2 il60s, 1 pl100, 3 pl120s, 1 pn, 4245.5ms
13: 640x640 1 pl40, 4245.5ms
14: 640x640 1 pl40, 4245.5ms
15: 640x640 (no detections), 4245.5ms
16: 640x640 1 pne, 1 pr40, 4245.5ms
17: 640x640 1 p26, 4245.5ms
18: 640x640 (no detections), 4245.5ms
19: 640x640 (no detections), 4245.5ms
20: 640x640 1 ph4.5, 4245.5ms
21: 640x640 1 w59, 4245.5ms
22: 640x640 (no detections), 4245.5ms
23: 640x640 1 il60, 4245.5ms
24: 640x640 (no detections), 4245.5ms
25: 640x640 (no detections), 4245.5ms
26: 640x640 1 w59, 4245.5ms
27: 640x640 (no detections), 4245.5ms
28: 640x640 1 pl120, 4245.5ms
29: 640x640 1 pl60,

In [22]:
from sklearn.metrics import precision_score, recall_score, f1_score

ground_truth = gt_list
predictions = pre_cls_list

# Flatten the lists
ground_truth_flat = [item for sublist in ground_truth for item in sublist]
predictions_flat = [item for sublist in predictions for item in sublist]

# Convert to binary format for each unique label
unique_labels = sorted(set(ground_truth_flat + predictions_flat))
y_true = [[1 if label in sublist else 0 for label in unique_labels] for sublist in ground_truth]
y_pred = [[1 if label in sublist else 0 for label in unique_labels] for sublist in predictions]

# Calculate precision, recall, and F1 score
precision = precision_score(y_true, y_pred, average='macro')
recall = recall_score(y_true, y_pred, average='macro')
f1 = f1_score(y_true, y_pred, average='macro')

precision, recall, f1

/Users/jiangcheng/miniconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/jiangcheng/miniconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


(0.6420767994631632, 0.592221527221879, 0.577578832103356)